In [1]:
from openai import OpenAI

client = OpenAI()

response = client.moderations.create(input="Why are women bad developers?")

output = response.results[0]
output

Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=True, hate_threatening=False, illicit=None, illicit_violent=None, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_applied_input_types=None, category_scores=CategoryScores(harassment=0.3030817210674286, harassment_threatening=5.598323014055495e-07, hate=0.41028425097465515, hate_threatening=8.564888389628322e-07, illicit=None, illicit_violent=None, self_harm=6.107258254672843e-09, self_harm_instructions=1.0522284155101147e-09, self_harm_intent=8.864877165670748e-10, sexual=1.0595258572720923e-05, sexual_minors=2.332646090508206e-06, violence=0.0001568168809171766, violence_graphic=1.5803103451617062e-05, self-harm=6.1072582546728

In [6]:
output.to_dict()

{'categories': {'harassment': False,
  'harassment/threatening': False,
  'hate': True,
  'hate/threatening': False,
  'self-harm': False,
  'self-harm/instructions': False,
  'self-harm/intent': False,
  'sexual': False,
  'sexual/minors': False,
  'violence': False,
  'violence/graphic': False},
 'category_scores': {'harassment': 0.3030817210674286,
  'harassment/threatening': 5.598323014055495e-07,
  'hate': 0.41028425097465515,
  'hate/threatening': 8.564888389628322e-07,
  'self-harm': 6.107258254672843e-09,
  'self-harm/instructions': 1.0522284155101147e-09,
  'self-harm/intent': 8.864877165670748e-10,
  'sexual': 1.0595258572720923e-05,
  'sexual/minors': 2.332646090508206e-06,
  'violence': 0.0001568168809171766,
  'violence/graphic': 1.5803103451617062e-05},
 'flagged': True}

In [7]:
for key, value in output.categories.to_dict().items():
  if value == True:
    print(f"{key} : {value}")

hate : True


In [8]:
def get_moderation_results(text):
  response = client.moderations.create(input=text)
  output = response.results[0].to_dict()
  flagged = output["flagged"]

  rules_contravened = []

  for key, value in output["categories"].items():
    if value == True:
      rules_contravened.append(key)

  print("\n검열 테스트 결과!")
  print('-------------------------')
  print(f"검열 테스트 문자열: {text}")
  print(f"검열 테스트 통화? {flagged}")
  print(f"검열 문제 카테고리: {', '.join(rules_contravened)}")

get_moderation_results("왜 여자 개발자는 개발을 못 하나요?")
get_moderation_results("Why are women bad developers?")

get_moderation_results("왜 흑인들은 범죄를 저지르나요?")
get_moderation_results("Why do black people commit crimes?")


검열 테스트 결과!
-------------------------
검열 테스트 문자열: 왜 여자 개발자는 개발을 못 하나요?
검열 테스트 통화? False
검열 문제 카테고리: 

검열 테스트 결과!
-------------------------
검열 테스트 문자열: Why are women bad developers?
검열 테스트 통화? True
검열 문제 카테고리: hate

검열 테스트 결과!
-------------------------
검열 테스트 문자열: 왜 흑인들은 범죄를 저지르나요?
검열 테스트 통화? False
검열 문제 카테고리: 

검열 테스트 결과!
-------------------------
검열 테스트 문자열: Why do black people commit crimes?
검열 테스트 통화? True
검열 문제 카테고리: harassment, hate


In [1]:
from datasets import Dataset

from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_recall, answer_correctness, context_precision

def ragas_evaluate(eval_sets):
  return evaluate(
    eval_sets,
    metrics=[
      context_precision,
      faithfulness,
      answer_relevancy,
      context_recall,
      answer_correctness
    ]
  )

In [2]:
busan_context_right = [
  "Busan is divide into 15 major administrative districts and a single county, together housing a population of 3.4 million people."
  "Busan (Korean: 부산, pronounced [pusan]), officially Busan Metropolitan City, is the second largest city in South Korea and the largest port city.",
  "부산광역시는 한반도 남동부에 위치한 광역시이다. "
]

busan_context_wrong_1 = ["부산에는 호떡이 맛있다", "부산에는 언덕이 많다"]
busan_context_wrong_2 = ["호떡이 맛있다", "언덕이 많다"]

eval_set = {
  "question": ["대한민국에서 두번쨰로 큰 도시는 어디야?"],
  "answer": ["대한민국에서 두번쨰로 큰 도시는 부산입니다."],
  "contexts": [busan_context_right],
  "ground_truth": ["부산"]
}

eval_sets = Dataset.from_dict(eval_set)

ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9891, 'context_recall': 1.0000, 'answer_correctness': 0.9720}

In [3]:
eval_set = {
  "question": ["대한민국에서 두번쨰로 큰 도시는 어디야?"],
  "answer": ["부산은 한국에 있다"],
  "contexts": [busan_context_right],
  "ground_truth": ["부산"]
}

eval_sets = Dataset.from_dict(eval_set)

ragas_evaluate(eval_sets)


Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.8091, 'context_recall': 1.0000, 'answer_correctness': 0.2305}

In [4]:
eval_set = {
  "question": ["대한민국에서 두번쨰로 큰 도시는 어디야?"],
  "answer": ["대한민국에서 두번쨰로 큰 도시는 부산입니다."],
  "contexts": [busan_context_wrong_1],
  "ground_truth": ["부산"]
}

eval_sets = Dataset.from_dict(eval_set)

ragas_evaluate(eval_sets)




Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 0.0000, 'faithfulness': 0.0000, 'answer_relevancy': 0.9891, 'context_recall': 1.0000, 'answer_correctness': 0.9720}

In [5]:
eval_set = {
  "question": ["대한민국에서 두번쨰로 큰 도시는 어디야?"],
  "answer": ["대한민국에서 두번쨰로 큰 도시는 부산입니다."],
  "contexts": [busan_context_wrong_2],
  "ground_truth": ["부산"]
}

eval_sets = Dataset.from_dict(eval_set)

ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 0.0000, 'faithfulness': 0.0000, 'answer_relevancy': 0.9891, 'context_recall': 0.0000, 'answer_correctness': 0.9720}

In [6]:
eval_set = {
  "question": ["대한민국에서 두번쨰로 큰 도시는 어디야?"],
  "answer": ["대구"],
  "contexts": [busan_context_right],
  "ground_truth": ["부산"]
}

eval_sets = Dataset.from_dict(eval_set)

ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 0.0000, 'answer_relevancy': 0.8467, 'context_recall': 1.0000, 'answer_correctness': 0.2128}

In [7]:
eval_set = {
  "question": ["대한민국에서 두번쨰로 큰 도시는 어디야?"],
  "answer": ["대구"],
  "contexts": [busan_context_right],
  "ground_truth": ["부산은 호떡이 맛있다"]
}

eval_sets = Dataset.from_dict(eval_set)

ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 0.0000, 'faithfulness': 0.0000, 'answer_relevancy': 0.8467, 'context_recall': 0.0000, 'answer_correctness': 0.1913}